In [1]:
!pip install bs4
import bs4
from bs4 import BeautifulSoup
import re
import urllib.request as ur
import requests
import time
import random as ran
import sys
import pandas as pd
import sqlite3

     |████████████████████████████████| 115 kB 1.2 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=54edc321b537de7970b486627a0cd67d75b3fd0579eef77c97a256c921db9b8c
  Stored in directory: /root/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


**Below cell creates a database and tables "OFFERING_DETAILS" and "UNDERWRITER_DETAILS".** 
Note: Have to run this only once

In [2]:
con = sqlite3.connect('offering_fields.db')
cursor = con.cursor()
cursor.execute('''CREATE TABLE OFFERING_DETAILS
                (COMPANY_NAME text,CENTRAL_INDEX_KEY text,SECURITY_OFFERED text,NUM_OF_SECURITIES text,OFFFERING_DATE int, OFFERING_PRICE text, INTEREST_RATE list, CONVERTIBLE_FLAG text, ACCESSION_NUMBER text,OFFERING_AMOUNT list)''')
cursor.execute('''CREATE TABLE UNDERWRITER_DETAILS
                (ACCESSION_NUMBER text,COMPANY_NAME text,CENTRAL_INDEX_KEY text,UNDERWRITER_NAME list,SECURITY_NAME text,OFFFERING_DATE int,SECURITIY_ALLOCATIION_IN_SHARES text, SECURITY_ALLOCATION_IN_$ list)''')
con.close()

**The following code is for extracting offering details. It is executed multiple times with different HTML and TXT files.**

In [75]:
"""Offering Details"""
html = '../input/htmltxt/424_TMUS.htm'
txt ='../input/htmltxt/424_TMUS.txt'
#A function to extract unique elements in a list
def unique(l):
    u_l = []
    for x in l:
        #print(x)
        if x not in u_l:
            u_l.append(x)
    return u_l


#txt file scraping. Scraping the required details from the TXT file.
with open(txt,'r') as text:
    for line in text:
        if 'ACCESSION NUMBER' in line:
            accession_number = line.partition('NUMBER:')[2]
            accession_number = accession_number.strip('\t')
            accession_number = accession_number.strip('\n')
        if 'FILED AS OF DATE' in line:
            filed = re.findall(r'[0-9]+',line) 
            filed_date = int(filed[0])
        if 'COMPANY CONFORMED NAME' in line:
            company_name = line.partition('NAME:')[2]
            company_name = company_name.strip('\t')
            company_name = company_name.strip('\n')
        if 'CENTRAL INDEX KEY' in line:
            central_index_key = re.findall(r'[0-9]+',line)
            break

        
#Creating parse tree unsing BeautifulSoup
with open(html,encoding='cp1252') as fp:
    soup = BeautifulSoup(fp,'html.parser')
#soup.decode()
para =soup.find_all('p')           

#....................................................................................................................................................................           
#To check Notes or not
interest=[]
offering_amount = []
    #This part of the code mainly focus on the central headings of the HTML which are under 'p' tag. 
    #For many files this was the best way to check whether a given paper is NOTES or NOT. Used keywords 'Notes due' and general forms of extracting 
    #[0-9]+.[0-9]+\%.*Notes.due.[0-9]+ and \$[0-9]+.*
for i in range (0,len(para)):
    if 'Notes due' in para[i].get_text():
        #print('a')
        interest += re.findall(r'[0-9]+.[0-9]+\%.*Notes.due.[0-9]+',para[i].get_text())
        offering_amount += re.findall(r'\$[0-9]+.*',para[i].get_text())

      #This is similar to the above code but the text we want is in the tag 'div',class_= 'h1'
if not len(offering_amount) and not len(interest):
    div = soup.find_all('div',class_= 'h1')
    for j in range (0,len(div)):
        if 'Notes due' in div[j].get_text():
            interest += re.findall(r'[0-9]+.[0-9]+\%.*Notes.due.[0-9]+',div[j].get_text())
            offering_amount += re.findall(r'\$[0-9]+.*',div[j].get_text())
interest = unique(interest)
offering_amount = unique(offering_amount)


    #This is similar to the above code but the text we want is in the tag 'p', align='CENTER'. 
if not len(offering_amount):
    center = soup.find_all('p', align='CENTER')
    for i in range (0,len(center)):
        offering_amount += re.findall(r'\$[0-9]+.*',center[i].get_text())
    offering_amount = unique(offering_amount) 
#..........................................................................................................................................................................
    
#Classifying Security offered as NOTES or SHARES
if len(interest) and len(offering_amount):
    security_offered = 'Notes'
    limit = min(len(offering_amount),len(interest))
    #print(limit)
    for i in range (0,limit):
        interest[i] = interest[i].split(" ")[0]
        offering_amount[i] = offering_amount[i].split(" ")[0]
        
else:
    security_offered = 'Shares'

#................................................................................................................................................................................    
    
#covertible flag check
if security_offered == 'Notes':
    convertible=[]
    for i in range (1,len(para)):
        if 'Convertible Senior Notes due' in para[i].get_text():
            convertible = re.findall(r'Convertible Senior Notes due [0-9]+',para[i].get_text())
    if len(convertible):
        covertible_flag = 'Yes'
    else:
        covertible_flag = 'No'
        
#................................................................................................................................................................................           
#For shares
ADS = 0
if security_offered == 'Shares':
    offering =[]
    num_of =[]
    offering_price = 0
    num_of_shares = 0
    #This part of the code mainly focus on the paragraphs of the HTML which are under 'p' tag. 
    #For many files this was the best way to extract the details in SHARES. Used keywords 'Notes due' and general forms of extracting 
    #'offering price of $' and general forms of extracting 'offering price of.* \$[0-9]+.[0-9]+ per share' and 'of [0-9]+.* shares'
    for i in range (0,len(para)):
        if 'offering price of $' in para[i].get_text():
            offering += re.findall(r'offering price of.* \$[0-9]+.[0-9]+ per share',para[i].get_text())
            num_of += re.findall(r'of [0-9]+.* shares',para[i].get_text())
        offering = unique(offering)
        num_of = unique(num_of) 
        

    #similar to the above code. A slight difference in the general form i.e., 'offering price of.* \$[0-9]+.[0-9]+ per ADS' and 'of [0-9]+.[0-9]+.* ADSs'
    if not len(offering) and not len(num_of):
        for i in range (0,len(para)):
            if 'offering price of $' in para[i].get_text():
                offering += re.findall(r'offering price of.* \$[0-9]+.[0-9]+ per ADS',para[i].get_text())
                num_of += re.findall(r'of [0-9]+.[0-9]+.* ADSs',para[i].get_text())
        offering = unique(offering)
        num_of = unique(num_of) 
        if len(offering) and len(num_of):
            ADS = 1
        
        
    #This part of the code mainly focus on the paragraphs of the HTML which are under 'h1' tag, which are in the center of the page, 
    #incase if the above two codes failed to extract the details
    if not len(offering) and not len(num_of):
        h1 = soup.find_all('h1')
        for i in range (0,len(h1)):
            if 'Shares' in h1[i].get_text():
                num_of += re.findall(r'[0-9]+.* Shares',h1[i].get_text())
                
            if 'per share' in h1[i].get_text():
                offering += re.findall(r'\$[0-9]+.[0-9]+ per share',h1[i].get_text())
        offering = unique(offering)
        num_of = unique(num_of) 
        
        
    #Converting the the extracted data from structure to int values to calculate the offering_amount
    if len(offering) and len(num_of):
        value1 = re.findall(r'[0-9]+',offering[0])
        value2 = re.findall(r'[0-9]+',num_of[0])

        
        offering_price = int(value1[0]) + int(value1[1])/pow(10,len(value1[1]))
        num_of_shares = 0
        for i in range (0,len(value2)):
            num_of_shares += int(value2[i])*pow(10,3*(len(value2)-i-1))
        offering_amount = offering_price*num_of_shares    

In [76]:
"""Underwriter Details"""
#Underwriter
t = soup.find_all('table')
u1 =[]
u2 = []
#The below code checks if there are any underwriter names present in the HTML tables.
for x in t:
    u1 += re.findall(r'Underwriter',x.get_text())
    u2 += re.findall(r'Name',x.get_text())
u1 = unique(u1)
u2 = unique(u2)
flag =0
underwriter = []
#if underwriter names are present in the HTML tables this part will extract the underwriter names from tables
if bool(len(u1) or len(u2)):
    if len(u1):
        tables = pd.read_html(html, match ='Underwriter')
    if len(u2):
        tables = pd.read_html(html, match ='Name')
    for i in range (0,len(tables[0])):
        if tables[0][0][i] == 'Total':
            break
        if flag:
            if type(tables[0][0][i]) == str and not tables[0][0][i] == '\u200b':
                #print(tables[0][0][i])
                #print(type(tables[0][0][i]))
                underwriter.append(tables[0][0][i])
        if tables[0][0][i] == 'Underwriter'or tables[0][0][i] == 'Underwriters' or tables[0][0][i] == 'Name' or tables[0][0][i] == 'Names':
            flag = 1
            
#This is a special case if the given shares are "ADS" to extract the Underwriter name
if ADS and not len(underwriter):
    tables = pd.read_html(html, match ='^Depositary')
    for i in range (0,len(tables[0])):
        if tables[0][0][i] == 'Depositary':
            underwriter.append(tables[0][2][i])

#This is also a special case if the underwriter name is not present in tables but present in between text.           
if not len(underwriter):
    for i in range (0,len(para)):
        if 'Securities Act' in para[i].get_text():
            if re.search(r'.* be deemed to be an',para[i].get_text()):
                underwriter = para[i].get_text()
    if(len(underwriter)):    
        underwriter = underwriter.split('deemed to be')[0]
        if len(underwriter.split('may be')) >= 2:
            underwriter = underwriter.split('may')[0]
        if len(underwriter.split('will be')) >= 2:
            underwriter = underwriter.split('will')[0]

        underwriter = underwriter.split(',')[len(underwriter.split(',')) -1]
        underwriter=underwriter.strip('\xa0')
        underwriter=underwriter.strip('\t')
        underwriter = underwriter.split('\n')
        if len(underwriter) >1:
            underwriter = ["".join(underwriter)]

#There is paper without underwriters.    
if not len(underwriter):
    underwriter = ['NA']

**storing the extracted values into the table.** Note executing multiples times leads to duplicates

In [77]:
if security_offered == 'Notes':
    interest_rate = []
    offering_Amount =[]
    for i in range (0,limit):
        interest_rate.append(interest[i])
    for i in range (0,limit):
        offering_Amount.append(offering_amount[i])
    num_of_securities = 'NA'
    offering_Price = 'NA'
    
else:
    num_of_securities = str(num_of_shares)
    offering_Price = '$'+str(offering_price)+' per share'
    interest_rate = ['NA']
    convertible_flag = 'NA'
    offering_Amount = '$'+str(offering_amount)
    
con = sqlite3.connect('./offering_fields.db')
con.execute("INSERT INTO OFFERING_DETAILS VALUES(?,?,?,?,?,?,?,?,?,?)",(str(company_name),str(central_index_key),str(security_offered),str(num_of_securities),int(filed_date),str(offering_Price),str(interest_rate),str(covertible_flag),str(accession_number),str(offering_Amount)))
con.execute("INSERT INTO UNDERWRITER_DETAILS VALUES(?,?,?,?,?,?,?,?)",(str(accession_number),str(company_name),str(central_index_key),str(underwriter),str(security_offered),int(filed_date),str(num_of_securities),str(offering_Amount)))
con.commit()
con.close()
    

**printing the extracted details to cross check**

In [78]:
print("Offering Details:")
print("Company Name:", company_name)
print("Central Index Key:", central_index_key)
print("Security Offered:", security_offered)
print("Number of Securities Offered:", num_of_securities)
print("Offering Date:", filed_date)
print("Offering Price:", offering_Price)
print("Interest Rate:", interest_rate) 
print("Convertible Flag:", covertible_flag)
print("ACCESSION NUMBER:", accession_number)
print("Offering Amount", offering_Amount)
print('\n\n')   
print("Underwriter Details:")
print("Accession Number:", accession_number)
print("Company Name:", company_name)
print("Central Index Key:", central_index_key)
print("Underwriter Name:", underwriter)
print("Security Name:", security_offered)
print("Offering Date:", filed_date)
print("Security Allocation in Shares:", num_of_securities)
print("Security Allocation in $:", offering_Amount)

Offering Details:
Company Name: T-Mobile US, Inc.
Central Index Key: ['0001283699']
Security Offered: Notes
Number of Securities Offered: NA
Offering Date: 20210113
Offering Price: NA
Interest Rate: ['2.250%', '2.625%', '2.875%']
Convertible Flag: No
ACCESSION NUMBER: 0001140361-21-001067
Offering Amount ['$1,000,000,000', '$1,000,000,000', '$1,000,000,000']



Underwriter Details:
Accession Number: 0001140361-21-001067
Company Name: T-Mobile US, Inc.
Central Index Key: ['0001283699']
Underwriter Name: ['Deutsche Bank Securities Inc.', 'Citigroup Global Markets Inc.', 'Credit Suisse Securities (USA) LLC', 'Goldman Sachs & Co. LLC', 'Barclays Capital Inc.', 'J.P. Morgan Securities LLC', 'Morgan Stanley & Co. LLC', 'RBC Capital Markets, LLC', 'Academy Securities, Inc.', 'C.L. King & Associates, Inc.', 'Great Pacific Securities', 'Mischler Financial Group, Inc.']
Security Name: Notes
Offering Date: 20210113
Security Allocation in Shares: NA
Security Allocation in $: ['$1,000,000,000', '$1

**Printing the values in tables**

In [79]:
con = sqlite3.connect('./offering_fields.db')
for row in con.execute("SELECT COMPANY_NAME,CENTRAL_INDEX_KEY,SECURITY_OFFERED,NUM_OF_SECURITIES,OFFFERING_DATE,OFFERING_PRICE,INTEREST_RATE,CONVERTIBLE_FLAG,ACCESSION_NUMBER,OFFERING_AMOUNT from OFFERING_DETAILS"):
    print("Offering Details:")
    for i in range (0,10):
        print(row[i])
    print('\n')
print('-----------------------------------------------------------------------------------------------------------------------------')
for row in con.execute("SELECT ACCESSION_NUMBER,COMPANY_NAME,CENTRAL_INDEX_KEY,UNDERWRITER_NAME ,SECURITY_NAME,OFFFERING_DATE,SECURITIY_ALLOCATIION_IN_SHARES, SECURITY_ALLOCATION_IN_$ from UNDERWRITER_DETAILS"):
    print("Underwriter Details:")
    for j in range (0,8):
        print(row[j])
    print('\n')

Offering Details:
INTERNATIONAL BUSINESS MACHINES CORP
['0000051143']
Notes
NA
20200504
NA
['1.700%', '1.950%', '2.850%', '2.950%']
No
0001047469-20-002772
['$1,250,000,000', '$1,350,000,000', '$650,000,000', '$750,000,000']


Offering Details:
Tesla, Inc.
['0001318605']
Shares
7791074
20201208
$641.76 per share
['NA']
No
0001193125-20-312218
$4999999650.24


Offering Details:
Zoom Video Communications, Inc.
['0001585521']
Shares
5147059
20210113
$340.0 per share
['NA']
No
0001193125-21-008153
$1750000060.0


Offering Details:
AerCap Holdings N.V.
['0001378789']
Notes
NA
20210107
NA
['1.750%']
No
0001193125-21-004217
['$1,000,000,000']


Offering Details:
Golden Minerals Co
['0001011509']
Shares
3153808
20180509
$0.41 per share
['NA']
No
0001047469-18-003625
$1293061.28


Offering Details:
Auris Medical Holding Ltd.
['0001601936']
Shares
2000000
20210115
$4.0 per share
['NA']
No
0001213900-21-002553
$8000000.0


Offering Details:
Global Medical REIT Inc.
['0001533615']
Shares
3500000
2

**This cell is to delete the tables in the database in case there is an error in the stored values.** Note:Execute only when the tables are to be deleted

In [80]:
#conn = sqlite3.connect('offering_fields.db')
#c = conn.cursor()
#c.execute("DROP TABLE OFFERING_DETAILS")
#c.execute("DROP TABLE UNDERWRITER_DETAILS")
#conn.commit()
#conn.close